In [1]:
import torch
from torchvision import datasets, transforms

# Veri dönüşümleri
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# MNIST veri setini yükleme
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Veri yükleyicilerini oluşturma
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


100%|██████████| 9912422/9912422 [00:00<00:00, 408486060.56it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 90874488.99it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 100776548.20it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 14366914.61it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms


# Sinir ağı modeli
class ClassifierModule(nn.Module):
    def __init__(self):
        super(ClassifierModule, self).__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.softmax(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ClassifierModule().to(device)

# Loss fonksiyonu ve optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Eğitim döngüsü
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(dataloader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

# Evaluation fonksiyonu
def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(dataloader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            running_loss += loss.item()
            _, predicted = output.max(1)
            correct += predicted.eq(target).sum().item()
            total += target.size(0)
    return running_loss / len(dataloader), correct / total

# Model eğitimi
num_epochs = 20

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_accuracy = evaluate(model, test_loader, criterion, device)
    print(f"Epoch {epoch+1}/{num_epochs}: Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Epoch 1/20: Train Loss: 1.5659, Test Loss: 1.5185, Test Accuracy: 0.9432
Epoch 2/20: Train Loss: 1.5116, Test Loss: 1.5118, Test Accuracy: 0.9491
Epoch 3/20: Train Loss: 1.5049, Test Loss: 1.5016, Test Accuracy: 0.9596
Epoch 4/20: Train Loss: 1.5001, Test Loss: 1.5043, Test Accuracy: 0.9564
Epoch 5/20: Train Loss: 1.4973, Test Loss: 1.5115, Test Accuracy: 0.9496
Epoch 6/20: Train Loss: 1.4970, Test Loss: 1.4965, Test Accuracy: 0.9646
Epoch 7/20: Train Loss: 1.4961, Test Loss: 1.4949, Test Accuracy: 0.9663
Epoch 8/20: Train Loss: 1.4970, Test Loss: 1.5205, Test Accuracy: 0.9399
Epoch 9/20: Train Loss: 1.4949, Test Loss: 1.4922, Test Accuracy: 0.9685
Epoch 10/20: Train Loss: 1.4928, Test Loss: 1.5011, Test Accuracy: 0.9594
Epoch 11/20: Train Loss: 1.4949, Test Loss: 1.4970, Test Accuracy: 0.9639
Epoch 12/20: Train Loss: 1.4917, Test Loss: 1.5203, Test Accuracy: 0.9403
Epoch 13/20: Train Loss: 1.4929, Test Loss: 1.4973, Test Accuracy: 0.9638
Epoch 14/20: Train Loss: 1.4910, Test Loss: 1.5

In [ ]:
import torch
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix
import numpy as np

# Test veri setini tahmin etme
def predict(model, dataloader, device):
    model.eval()
    predictions = []
    true_labels = []
    with torch.no_grad():
        for data, target in dataloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(target.cpu().numpy())
    return predictions, true_labels

# Test verilerini tahmin et
test_predictions, test_labels = predict(model, test_loader, device)

# Sınıf etiketlerini tanımla
class_labels = [str(i) for i in range(10)]

# Confusion Matrix'i hesapla
cm = confusion_matrix(test_labels, test_predictions)

# Confusion Matrix'i yazdır
print("Confusion Matrix:")
print("     ", end="")
for label in class_labels:
    print(f"{label:<5s}", end="")
print()
for i, row in enumerate(cm):
    print(f"{class_labels[i]:<5s}", end="")
    for value in row:
        print(f"{value:<5d}", end="")
    print()

In [ ]:
import torch
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix

# Test veri setini tahmin etme
def predict(model, dataloader, device):
    model.eval()
    predictions = []
    true_labels = []
    with torch.no_grad():
        for data, target in dataloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(target.cpu().numpy())
    return predictions, true_labels

# Test verilerini tahmin et
test_predictions, test_labels = predict(model, test_loader, device)

# Confusion Matrix'i hesapla
cm = confusion_matrix(test_labels, test_predictions)
print("Confusion Matrix:")
print(cm)


Confusion Matrix:
[[ 968    2    1    0    0    1    4    3    1    0]
 [   0 1126    2    2    0    1    3    0    1    0]
 [   4    1 1008    6    1    1    3    7    1    0]
 [   0    1   13  953    1   11    1   10    5   15]
 [   1    3    8    0  915    2   16    1    4   32]
 [   3    1    0    3    1  860    9    5    2    8]
 [   5    3    0    0    1    9  938    1    1    0]
 [   1   11   17    0    0    0    1  983    0   15]
 [   5    3   21   13    7   10    9   10  869   27]
 [   4    6    2    7   11    3    2    5    1  968]]


In [ ]:
import torch
import pandas as pd

# Test veri setini tahmin etme
def predict(model, dataloader, device):
    model.eval()
    predictions = []
    true_labels = []
    with torch.no_grad():
        for data, target in dataloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(target.cpu().numpy())
    return predictions, true_labels

# Test verilerini tahmin et
test_predictions, test_labels = predict(model, test_loader, device)

# True/False değerlerini hesapla
true_values = [1 if pred == label else 0 for pred, label in zip(test_predictions, test_labels)]

# Doğru ve yanlış tahminlerin veri çerçevesini oluştur
data = {'Tahmin': test_predictions, 'Gerçek': test_labels, 'Doğru': true_values}
df = pd.DataFrame(data)

# Tabloyu ekrana yazdır
print(df)

      Tahmin  Gerçek  Doğru
0          7       7      1
1          2       2      1
2          1       1      1
3          0       0      1
4          4       4      1
...      ...     ...    ...
9995       2       2      1
9996       3       3      1
9997       4       4      1
9998       5       5      1
9999       6       6      1

[10000 rows x 3 columns]


In [ ]:
import torch

# Test veri setini tahmin etme
def predict(model, dataloader, device):
    model.eval()
    predictions = []
    true_labels = []
    with torch.no_grad():
        for data, target in dataloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(target.cpu().numpy())
    return predictions, true_labels

# Test verilerini tahmin et
test_predictions, test_labels = predict(model, test_loader, device)

# True/False değerlerini hesapla
true_values = [1 if pred == label else 0 for pred, label in zip(test_predictions, test_labels)]

# Doğru ve yanlış sayılarını hesapla
true_count = sum(true_values)
false_count = len(true_values) - true_count

print("Doğru tahmin sayısı:", true_count)
print("Yanlış tahmin sayısı:", false_count)


Doğru tahmin sayısı: 9588
Yanlış tahmin sayısı: 412


In [3]:
def apply_lorentz_transformation(data):
    # Lorentz dönüşümü uygulama kodu
    beta = 0.99
    gamma = 1 / (torch.sqrt(torch.tensor(1 - beta**2)) if beta < 1 else float('inf'))  # Lorentz faktörü

    data_tensor = torch.tensor(data, dtype=torch.float32)  # Data'yı Tensor'a dönüştür
    transformed_data = gamma * (data_tensor - beta * data_tensor)
    return transformed_data

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms


# Eğitim döngüsü
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(dataloader):
        data, target = data.to(device), target.to(device)

        # Lorentz dönüşümü uygulama
        data = apply_lorentz_transformation(data)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

# Evaluation fonksiyonu
def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(dataloader):
            data, target = data.to(device), target.to(device)

            # Lorentz dönüşümü uygulama
            data = apply_lorentz_transformation(data)

            output = model(data)
            loss = criterion(output, target)
            running_loss += loss.item()
            _, predicted = output.max(1)
            correct += predicted.eq(target).sum().item()
            total += target.size(0)
    return running_loss / len(dataloader), correct / total

# Model eğitimi
num_epochs = 20

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_accuracy = evaluate(model, test_loader, criterion, device)
    print(f"Epoch {epoch+1}/{num_epochs}: Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

<ipython-input-3-0581cc07d466>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data_tensor = torch.tensor(data, dtype=torch.float32)  # Data'yı Tensor'a dönüştür


Epoch 1/20: Train Loss: 1.4803, Test Loss: 1.4843, Test Accuracy: 0.9780
Epoch 2/20: Train Loss: 1.4757, Test Loss: 1.4831, Test Accuracy: 0.9783
Epoch 3/20: Train Loss: 1.4742, Test Loss: 1.4819, Test Accuracy: 0.9795
Epoch 4/20: Train Loss: 1.4732, Test Loss: 1.4817, Test Accuracy: 0.9788
Epoch 5/20: Train Loss: 1.4731, Test Loss: 1.4809, Test Accuracy: 0.9807
Epoch 6/20: Train Loss: 1.4723, Test Loss: 1.4816, Test Accuracy: 0.9798
Epoch 7/20: Train Loss: 1.4720, Test Loss: 1.4815, Test Accuracy: 0.9797
Epoch 8/20: Train Loss: 1.4712, Test Loss: 1.4805, Test Accuracy: 0.9809
Epoch 9/20: Train Loss: 1.4704, Test Loss: 1.4814, Test Accuracy: 0.9797
Epoch 10/20: Train Loss: 1.4696, Test Loss: 1.4829, Test Accuracy: 0.9784
Epoch 11/20: Train Loss: 1.4697, Test Loss: 1.4804, Test Accuracy: 0.9808
Epoch 12/20: Train Loss: 1.4697, Test Loss: 1.4787, Test Accuracy: 0.9825
Epoch 13/20: Train Loss: 1.4691, Test Loss: 1.4809, Test Accuracy: 0.9803
Epoch 14/20: Train Loss: 1.4685, Test Loss: 1.4

In [18]:
def apply_lorentz_transformation1(data):
    # Lorentz dönüşümü uygulama kodu
    #beta = 0.99
    #gamma = 1 / (torch.sqrt(torch.tensor(1 - beta**2)) if beta < 1 else float('inf'))  # Lorentz faktörü

    data_tensor = torch.tensor(data, dtype=torch.float32)  # Data'yı Tensor'a dönüştür
    transformed_data1 = data_tensor*0.05
    return transformed_data1

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms


# Eğitim döngüsü
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(dataloader):
        data, target = data.to(device), target.to(device)

        # Lorentz dönüşümü uygulama
        data = apply_lorentz_transformation1(data)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

# Evaluation fonksiyonu
def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(dataloader):
            data, target = data.to(device), target.to(device)

            # Lorentz dönüşümü uygulama
            data = apply_lorentz_transformation1(data)

            output = model(data)
            loss = criterion(output, target)
            running_loss += loss.item()
            _, predicted = output.max(1)
            correct += predicted.eq(target).sum().item()
            total += target.size(0)
    return running_loss / len(dataloader), correct / total

# Model eğitimi
num_epochs = 20

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_accuracy = evaluate(model, test_loader, criterion, device)
    print(f"Epoch {epoch+1}/{num_epochs}: Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

<ipython-input-18-30327b73da2e>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data_tensor = torch.tensor(data, dtype=torch.float32)  # Data'yı Tensor'a dönüştür


Epoch 1/20: Train Loss: 1.4779, Test Loss: 1.4834, Test Accuracy: 0.9775
Epoch 2/20: Train Loss: 1.4733, Test Loss: 1.4807, Test Accuracy: 0.9802
Epoch 3/20: Train Loss: 1.4721, Test Loss: 1.4815, Test Accuracy: 0.9795
Epoch 4/20: Train Loss: 1.4708, Test Loss: 1.4821, Test Accuracy: 0.9788
Epoch 5/20: Train Loss: 1.4702, Test Loss: 1.4818, Test Accuracy: 0.9792
Epoch 6/20: Train Loss: 1.4692, Test Loss: 1.4803, Test Accuracy: 0.9806
Epoch 7/20: Train Loss: 1.4700, Test Loss: 1.4815, Test Accuracy: 0.9796
Epoch 8/20: Train Loss: 1.4691, Test Loss: 1.4792, Test Accuracy: 0.9820
Epoch 9/20: Train Loss: 1.4680, Test Loss: 1.4826, Test Accuracy: 0.9781
Epoch 10/20: Train Loss: 1.4692, Test Loss: 1.4814, Test Accuracy: 0.9795
Epoch 11/20: Train Loss: 1.4687, Test Loss: 1.4798, Test Accuracy: 0.9812
Epoch 12/20: Train Loss: 1.4683, Test Loss: 1.4779, Test Accuracy: 0.9833
Epoch 13/20: Train Loss: 1.4676, Test Loss: 1.4798, Test Accuracy: 0.9811
Epoch 14/20: Train Loss: 1.4672, Test Loss: 1.4

In [ ]:
import torch
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix
import numpy as np

# Test veri setini tahmin etme
def predict(model, dataloader, device):
    model.eval()
    predictions = []
    true_labels = []
    with torch.no_grad():
        for data, target in dataloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(target.cpu().numpy())
    return predictions, true_labels

# Test verilerini tahmin et
test_predictions, test_labels = predict(model, test_loader, device)

# Sınıf etiketlerini tanımla
class_labels = [str(i) for i in range(10)]

# Confusion Matrix'i hesapla
cm = confusion_matrix(test_labels, test_predictions)

# Confusion Matrix'i yazdır
print("Confusion Matrix:")
print("     ", end="")
for label in class_labels:
    print(f"{label:<5s}", end="")
print()
for i, row in enumerate(cm):
    print(f"{class_labels[i]:<5s}", end="")
    for value in row:
        print(f"{value:<5d}", end="")
    print()

In [ ]:
import torch
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix

# Test veri setini tahmin etme
def predict(model, dataloader, device):
    model.eval()
    predictions = []
    true_labels = []
    with torch.no_grad():
        for data, target in dataloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(target.cpu().numpy())
    return predictions, true_labels

# Test verilerini tahmin et
test_predictions, test_labels = predict(model, test_loader, device)

# Confusion Matrix'i hesapla
cm = confusion_matrix(test_labels, test_predictions)
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[ 971    1    1    0    0    1    4    1    1    0]
 [   0 1126    3    0    0    1    5    0    0    0]
 [   3    3 1006    3    2    2    3    8    2    0]
 [   2    0    5  975    0   16    0    8    2    2]
 [   2    0    2    0  946    1    8    2    0   21]
 [   5    1    0    3    1  870    7    1    1    3]
 [   5    3    0    0    2    4  943    0    1    0]
 [   0    9   12    1    0    1    1 1000    0    4]
 [   9    1    3   10    5   19   13    4  906    4]
 [   3    6    1    9    8    9    2    7    2  962]]


In [ ]:
import torch
import pandas as pd

# Test veri setini tahmin etme
def predict(model, dataloader, device):
    model.eval()
    predictions = []
    true_labels = []
    with torch.no_grad():
        for data, target in dataloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(target.cpu().numpy())
    return predictions, true_labels

# Test verilerini tahmin et
test_predictions, test_labels = predict(model, test_loader, device)

# True/False değerlerini hesapla
true_values = [1 if pred == label else 0 for pred, label in zip(test_predictions, test_labels)]

# Doğru ve yanlış tahminlerin veri çerçevesini oluştur
data = {'Tahmin': test_predictions, 'Gerçek': test_labels, 'Doğru': true_values}
df = pd.DataFrame(data)

# Tabloyu ekrana yazdır
print(df)


      Tahmin  Gerçek  Doğru
0          7       7      1
1          2       2      1
2          1       1      1
3          0       0      1
4          4       4      1
...      ...     ...    ...
9995       2       2      1
9996       3       3      1
9997       4       4      1
9998       5       5      1
9999       6       6      1

[10000 rows x 3 columns]


In [ ]:
import torch

# Test veri setini tahmin etme
def predict(model, dataloader, device):
    model.eval()
    predictions = []
    true_labels = []
    with torch.no_grad():
        for data, target in dataloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(target.cpu().numpy())
    return predictions, true_labels

# Test verilerini tahmin et
test_predictions, test_labels = predict(model, test_loader, device)

# True/False değerlerini hesapla
true_values = [1 if pred == label else 0 for pred, label in zip(test_predictions, test_labels)]

# Doğru ve yanlış sayılarını hesapla
true_count = sum(true_values)
false_count = len(true_values) - true_count

print("Doğru tahmin sayısı:", true_count)
print("Yanlış tahmin sayısı:", false_count)


Doğru tahmin sayısı: 9705
Yanlış tahmin sayısı: 295
